## Runnable Interface 介绍与使用

为了尽可能简化创建自定义链的过程，Langchain 实现了一个 **[Runnable](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)** 协议。

许多 LangChain 组件都实现了 Runnable 协议，包括 chat models, LLMs, output parsers, retrievers, prompt templates等等。此外，还有几个用于处理可运行对象的[有用原语](https://python.langchain.com/v0.1/docs/expression_language/primitives/)。

Runnable 是一个标准接口，包括：

- stream：流式返回生成内容（chunk）
- invoke：对输入调用该链
- batch：对输入列表调用该链

不同组件的输入和输出类型有所差异:

| 组件    | 输入类型                                           | 输出类型           |
| ------------ | ----------------------------------------------------- | --------------------- |
| Prompt       | Dictionary                                            | PromptValue           |
| ChatModel    | Single string, list of chat messages or a PromptValue | ChatMessage           |
| LLM          | Single string, list of chat messages or a PromptValue | String                |
| OutputParser | The output of an LLM or ChatModel                     | Depends on the parser |
| Retriever    | Single string                                         | List of Documents     |
| Tool         | Single string or dictionary, depending on the tool    | Depends on the tool   |


所有 Runnable 对象都显式描述输入和输出 Schema，以检查输入和输出格式：

- input_schema：从Runnable的结构自动生成的输入Pydantic模型
- output_schema：从Runnable的结构自动生成的输出Pydantic模型 


### Input Schema

为了演示如何使用，下面我们创建一个超级简单的PromptTemplate + ChatModel链。

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_template("讲个关于 {topic} 的笑话吧")
chain = prompt | model

#### schema 方法

一个描述 Runnable 接受的输入的说明。这是根据任何 Runnable 结构动态生成的 Pydantic 模型。您可以调用 .schema() 来获取 `JSONSchema` 表示。

In [15]:
# 查看 Chain 的输入类型
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [3]:
# 查看 Prompt 的输入类型（Chain的输入从 Prompt 开始，因此输入类型一致）
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [4]:
# 查看 Model 的输入类型
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

### Output Schema

输出类型仍然可以调用 .schema() 来获取其 `JSONSchema` 表示。

In [5]:
# 查看 Chain 的输出类型
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'}},
   'required': ['name', 'args', 'id', 'error']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'proper

### Stream

使用 .stream() 方法查看（同步）流式输出结果

In [6]:
for s in chain.stream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

为什么程序员总是开玩笑？因为他们喜欢用幽默感来解决bug！

### Invoke
使用 .invoke() 方法单次（同步）调用

In [7]:
chain.invoke({"topic": "程序员"})

AIMessage(content='为什么程序员喜欢在厨房工作？\n\n因为他们喜欢处理bug！', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 22, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bc7e974c-91d5-4e26-9e17-1f8cab3f0e8a-0')

### Batch
使用 .batch() 方法（同步）批量调用

In [8]:
chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='为什么程序员总是喜欢猜谜语？\n\n因为他们喜欢解决问题！', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 22, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e43c25ed-8dc9-4d7c-97ea-063d3c3f34b2-0'),
 AIMessage(content='为什么产品经理喜欢用Excel表格来做需求分析？\n\n因为他们觉得Excel是需求分析的“表”哥！😄😄😄', response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 21, 'total_tokens': 70}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-67aec48b-e6a3-4871-a9ad-01806b574e92-0'),
 AIMessage(content='为了提高团队的工作效率，测试经理在团队会议上提出了一个新的规定：每个人都要在测试之前自己动手试一试。\n\n结果第二天，测试经理看到整个团队都在用力地拧着测试设备，他问其中一位成员：“你在干什么？”\n\n成员回答道：“老板，您说每个人都要在测试之前自己动手试一试，所以我们正在试着拧开这个设备，看看能不能在测试之前把它弄坏！”', response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 21, 'total_tokens': 178}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': Non

In [9]:
messages = chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

In [10]:
# 使用 StrOutputParser 来处理 Batch 批量输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

for idx, m in enumerate(messages):
    print(f"笑话{idx}:\n")
    print(output_parser.invoke(m))
    print("\n")

笑话0:

为什么程序员总是喜欢熬夜工作呢？因为他们认为夜晚是代码最安静的时候，可以更好地思考和创造！


笑话1:

一位产品经理走进一家餐馆，点了一道菜，却发现菜上的调料完全不对。他气愤地向服务员投诉，服务员笑着说：“对不起，这是我们的‘用户体验测试’菜单，我们需要您的反馈来改进。”产品经理无语地点点头，心想：“原来不只是我在做用户测试啊！”笑着接受了这个冷笑话。


笑话2:

为了测试一个新的笑话系统，测试经理和开发工程师走进了一个酒吧。测试经理说：“我要一杯啤酒，但请确保它没有任何bug。”开发工程师笑着说：“别担心，我已经进行了充分的单元测试和集成测试，这杯啤酒应该是没有bug的。”测试经理接过啤酒，喝了一口，然后说：“哎呀，这杯啤酒有一个致命的bug。”开发工程师惊讶地问：“什么bug？”测试经理回答：“它让我有点晕。”笑话系统:笑声响起。




## 异步操作

这些方法也有相应的异步方法，应与 `asyncio` 的 `await` 语法一起使用以进行并发操作：

- astream：异步地流式返回生成内容（chunk）
- ainvoke：异步地对输入调用该链
- abatch：异步地对输入列表调用该链
- astream_log: 在发生时会返回中间步骤，并且最终返回结果之外。
- astream_events: beta 流式传输事件，在 langchain-core 0.1.14 中引入


### Async Stream

In [11]:
async for s in chain.astream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

为什么程序员不能打电话给女朋友？

因为他们总是在一个死循环里！

### Async Invoke

In [12]:
await chain.ainvoke({"topic": "程序员"})

AIMessage(content='为什么程序员喜欢喝咖啡？\n因为咖啡因能让他们保持 “Java” 状态！', response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 22, 'total_tokens': 61}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-96669abb-d2f2-4fe2-a647-c60e4a43f740-0')

### Async Batch

In [13]:
await chain.abatch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='为什么程序员总是深情地与电脑对话？\n\n因为他们知道，如果电脑出了什么问题，唯一能回应他们的就是它自己了！😄', response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 22, 'total_tokens': 81}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-03804162-89e3-469f-9469-0db1a0496924-0'),
 AIMessage(content='为什么产品经理总是迟到？\n因为他们总是在等需求定稿！', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 21, 'total_tokens': 47}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f9079c6e-5647-4e61-8450-d7e7603d0a14-0'),
 AIMessage(content='为什么测试经理总是喜欢在周五下班前发布新的测试任务？因为他们喜欢给团队一个“周末惊喜”！😄', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 21, 'total_tokens': 72}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b685b84f-c475-4527-851c-15e794687570-0')]